##Import librerie

In [49]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

In [50]:
import sys
sys.path.append('./')

from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [51]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

Connessione al servizio Drive Colab

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Gestione dei Data Set di Training e Test

lettura e stampa del training set

In [53]:
dataset="/content/drive/My Drive/Machine Learning Project/monks/monks-3_train.csv"
df_train = pd.DataFrame(pd.read_csv(dataset))
df_train.head()

,classe,a1,a2,a3,a4,a5,a6
0,1,1,1,1,1,1,2
1,1,1,1,1,1,2,1
2,1,1,1,1,1,2,2
3,0,1,1,1,1,3,1
4,0,1,1,1,1,4,1


In [54]:
# label encoding the data
from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
  
df_train['a1']= le.fit_transform(df_train['a1'])
df_train['a2']= le.fit_transform(df_train['a2'])
df_train['a3']= le.fit_transform(df_train['a3'])
df_train['a4']= le.fit_transform(df_train['a4'])
df_train['a5']= le.fit_transform(df_train['a5'])
df_train['a6']= le.fit_transform(df_train['a6'])

In [55]:
y2 = pd.get_dummies(df_train.a1, prefix='a1')
y3 = pd.get_dummies(df_train.a2, prefix='a2')
y4 = pd.get_dummies(df_train.a3, prefix='a3')
y5 = pd.get_dummies(df_train.a4, prefix='a4')
y6 = pd.get_dummies(df_train.a5, prefix='a5')
y7 = pd.get_dummies(df_train.a6, prefix='a6')

df_train = df_train.drop("a1", axis=1)
df_train = df_train.drop("a2", axis=1)
df_train = df_train.drop("a3", axis=1)
df_train = df_train.drop("a4", axis=1)
df_train = df_train.drop("a5", axis=1)
df_train = df_train.drop("a6", axis=1)


frames = [df_train,y2,y3,y4,y5,y6,y7]

df_train = pd.concat(frames, axis=1, join='inner')
display(df_train)

,classe,a1_0,a1_1,a1_2,a2_0,a2_1,a2_2,a3_0,a3_1,a4_0,a4_1,a4_2,a5_0,a5_1,a5_2,a5_3,a6_0,a6_1
0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1
1,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0
2,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1
3,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
4,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1
118,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1
119,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,1,0
120,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1


lettura e stampa del test set

In [56]:
dataset="/content/drive/My Drive/Machine Learning Project/monks/monks-3_test.csv"
df_test = pd.DataFrame(pd.read_csv(dataset))
df_test.head()


,classe,a1,a2,a3,a4,a5,a6
0,1,1,1,1,1,1,1
1,1,1,1,1,1,1,2
2,1,1,1,1,1,2,1
3,1,1,1,1,1,2,2
4,1,1,1,1,1,3,1


In [57]:
# label encoding the data
from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
  
df_test['a1']= le.fit_transform(df_test['a1'])
df_test['a2']= le.fit_transform(df_test['a2'])
df_test['a3']= le.fit_transform(df_test['a3'])
df_test['a4']= le.fit_transform(df_test['a4'])
df_test['a5']= le.fit_transform(df_test['a5'])
df_test['a6']= le.fit_transform(df_test['a6'])

In [58]:
y2 = pd.get_dummies(df_test.a1, prefix='a1')
y3 = pd.get_dummies(df_test.a2, prefix='a2')
y4 = pd.get_dummies(df_test.a3, prefix='a3')
y5 = pd.get_dummies(df_test.a4, prefix='a4')
y6 = pd.get_dummies(df_test.a5, prefix='a5')
y7 = pd.get_dummies(df_test.a6, prefix='a6')

df_test = df_test.drop("a1", axis=1)
df_test = df_test.drop("a2", axis=1)
df_test = df_test.drop("a3", axis=1)
df_test = df_test.drop("a4", axis=1)
df_test = df_test.drop("a5", axis=1)
df_test = df_test.drop("a6", axis=1)


frames = [df_test,y2,y3,y4,y5,y6,y7]

df_test = pd.concat(frames, axis=1, join='inner')
display(df_test)

,classe,a1_0,a1_1,a1_2,a2_0,a2_1,a2_2,a3_0,a3_1,a4_0,a4_1,a4_2,a5_0,a5_1,a5_2,a5_3,a6_0,a6_1
0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0
1,1,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1
2,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0
3,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1
4,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1
428,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
429,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1
430,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0


In [59]:
from google.colab import drive
drive.mount('drive')

df_train.to_csv('/content/drive/My Drive/Machine Learning Project/monks/monks_encoded/monk3_encoded_train.csv')
!cp data.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
cp: cannot stat 'data.csv': No such file or directory


In [60]:
df_test.to_csv('/content/drive/My Drive/Machine Learning Project/monks/monks_encoded/monk3_encoded_test.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory
